## AutoEncoder

In [ ]:
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [3]:
dataset = pd.read_csv(
    "../Datasets/Household_Electric_Power_Consumption/household_power_consumption.csv",
    #parse_dates={'Datetime': ['Date', 'Time']},
    na_values='?')
dataset.dropna(inplace=True)

features = dataset[['Global_active_power',
                'Global_reactive_power',
                'Voltage',
                'Global_intensity',
                'Sub_metering_1',
                'Sub_metering_2',
                'Sub_metering_3']].astype(np.float32)

In [ ]:
scaler = StandardScaler()
x = scaler.fit_transform(features[['Global_reactive_power',
                'Voltage',
                'Global_intensity',
                'Sub_metering_1',
                'Sub_metering_2',
                'Sub_metering_3']])

y = scaler.fit_transform(features[['Global_active_power']])

X_normalized_tensor = torch.FloatTensor(x)


In [9]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, input_dim),
            nn.ReLU(True)
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:

# Hyperparameters
input_dim = x.shape[1]  # Original feature dimension (4 for Iris)
encoding_dim = 4  # You can adjust the size of the encoding
num_epochs = 1000
learning_rate = 0.05

# Create the autoencoder model
model = Autoencoder(input_dim, encoding_dim)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the autoencoder
for epoch in range(num_epochs):
    model.train()
    
    # Zero the gradients
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_normalized_tensor)
    
    # Compute the loss
    loss = criterion(outputs, X_normalized_tensor)
    
    # Backward pass
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    # Print the loss at every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Reconstruct the original data
model.eval()
with torch.no_grad():
    X_reconstructed_tensor = model(X_normalized_tensor)

X_reconstructed = X_reconstructed_tensor.numpy()

# Calculate MSE
mse = mean_squared_error(x, X_reconstructed)
print(f'Mean Squared Error: {mse}')

# Calculate R²
variance_original = np.var(x)
r_squared = 1 - (mse / variance_original)
print(f'R-squared: {r_squared}')

NameError: name 'X_normalized_tensor' is not defined